## slots, 槽

我们定义一个类以后, 可以对这个类绑定任何的属性和方法.

In [1]:
class Student(object):
    pass

In [2]:
A = Student()
A.name = 'Amy'
A.age = 16
A.score = 100

In [3]:
A.print_age = lambda self:print(self.age)

In [4]:
A.print_age(A)

16


但是这样的绑定只可以针对一个实例, 其他实例是没有的, 我们可以对类添加属性和方法

In [5]:
Student.print_age = lambda self:print(self.age)

In [6]:
B = Student()

注意这里调用的区别, 对实例绑定方法的时候, self是需要传入的, 而对类绑定方法就不需要传入self.

In [7]:
B.age = 17
B.print_age() 

17


如果我们希望对类限制属性, 只希望绑定特定的属性, 那么就需要使用slots

In [8]:
class Student(object):
    __slots__ = ('name', 'age') 

In [9]:
A = Student()

In [10]:
A.name = 'Amy'
A.age = 18

In [11]:
A.score = 100 # 只能访问slots中的属性

AttributeError: 'Student' object has no attribute 'score'

slots只对当前实例有效, 继承的子类就无效了

In [12]:
class BoyStudent(Student):
    pass

In [13]:
B = BoyStudent()

In [14]:
B.score = 100

## property

如果我们希望对于输入的属性进行判断, 例如不能输如一些非法变量, 我们可以这样设计

In [15]:
class Student(object):
    def set_score(self, score):
        if score < 0 or score >100:
            raise ValueError('score should be 0 ~ 100')
        else:
            self.score = score

In [16]:
A = Student()

In [17]:
A.set_score(999)

ValueError: score should be 0 ~ 100

In [18]:
A.set_score(97)

In [19]:
A.score

97

但是为了装逼, 还可以使用property装饰器, 用来更方便的实现上述功能, 直接在赋值的时候检查

In [20]:
class Student(object):
    @property
    def score(self):
        return self.__score
    @score.setter
    def score(self, score):
        if score > 100 or score < 0:
            raise ValueError('score should be 0 ~ 100')
        else:
            self.__score = score

In [21]:
A = Student()

In [22]:
A.score = 199

ValueError: score should be 0 ~ 100

In [23]:
A.score = 89

In [24]:
class Student(object):
    @property
    def score(self):
        return self.__score
    @score.setter
    def score(self, score):
        if score > 100 or score < 0:
            raise ValueError('score should be 0 ~ 100')
        else:
            self.__score = score
    @property
    def substract(self):
        return 100-self.__score

In [25]:
A = Student()
A.score = 78
A.substract

22

In [26]:
A.substract = 10 ## 只有@property 修饰的属性只有读， 没有写， 除非定义一个setter的函数

AttributeError: can't set attribute

## 定制类

我们可以给类定制一些特殊的方法， 用以修饰类

### `__str__`

In [27]:
class Student(object):
    def __init__(self, name):
        self.name = name
        
A = Student('Amy')

In [28]:
print(A) # 这里的输出不好看(大雾), 所以我们可以在str方法上修改, 以便看出student类中的信息

In [29]:
class Student(object):
    def __init__(self, name):
        self.name = name
    def __str__(self):
        return 'Student object (name: {})'.format(self.name)
A = Student('Amy')

In [30]:
print(A)

Student object (name: Amy)


In [31]:
A # 但是A 还是这样, 这是由于print调用的是str, 而直接输出调用的是repr

In [32]:
class Student(object):
    def __init__(self, name):
        self.name = name
    def __str__(self):
        return 'Student object (name: {})'.format(self.name)
    __repr__ = __str__
A = Student('Amy')

In [33]:
A

Student object (name: Amy)

### `__iter__`

如果希望一个类拥有迭代的功能, 那么就需要有iter方法

In [34]:
class Fib(object):
    def __init__(self):
        self.a, self.b = 0, 1
    def __iter__(self):
        return self
    def __next__(self):
        self.a, self.b = self.b, self.a + self.b
        return self.a

In [35]:
for i in Fib():
    if i < 100:
        print(i)
    else:
        break

1
1
2
3
5
8
13
21
34
55
89


我们还可以使用索引来访问， 这就需要getitem方法

In [36]:
class Fib(object):
    def __init__(self):
        self.a, self.b = 0, 1
    def __iter__(self):
        return self
    def __next__(self):
        self.a, self.b = self.b, self.a + self.b
        return self.a
    def __getitem__(self, n):
        a, b = 0, 1
        for i in range(n):
            a, b = b, a + b
        return a

In [37]:
A = Fib()

In [38]:
A[10]

55

但是这样不可以访问切片, 所以我们还需要定义切片的操作

In [39]:
class Fib(object):
    def __init__(self):
        self.a, self.b = 0, 1
    def __iter__(self):
        return self
    def __next__(self):
        self.a, self.b = self.b, self.a + self.b
        return self.a
    def __getitem__(self, n):
        a, b = 0, 1
        if isinstance(n, str):
            for i in range(n):
                a, b = b, a + b
            return a
        elif isinstance(n, slice):
            a, b = 0, 1
            start, stop, step = n.start, n.stop, n.step
            l = []
            for i in range(stop):
                a, b = b, a + b
                l.append(a)
            return l[start:stop:step]

In [40]:
a = Fib()

In [41]:
a[1:10:2]# 这里的操作还不可以实现负数索引的访问

[1, 3, 8, 21, 55]

### `__getattr__`

一般如果访问不存在的属性, 那么就会报错， 使用getattr可以避免报错, 返回默认值

In [42]:
class Student(object):
    def __getattr__(self, attr):
        if not hasattr(Student, attr):
            self.attr = 100
            return self.attr
        else:
            return self.attr

In [43]:
A = Student()

In [44]:
A.age

100

In [45]:
class Student(object):
    def __getattr__(self, attr):
        if attr == 'score':
            self.attr = 100
        else:
            raise AttributeError('\'Student\' has no attribute {}'.format(attr))
        return self.attr

In [46]:
A = Student()

In [47]:
A.name

AttributeError: 'Student' has no attribute name

In [48]:
A.score

100

In [49]:
class Chain(object):

    def __init__(self, path=''):
        self._path = path

    def __getattr__(self, path):
        return Chain('%s/%s' % (self._path, path))

    def __str__(self):
        return self._path

    __repr__ = __str__

In [50]:
Chain().status.user.timeline.list

/status/user/timeline/list

In [51]:
class Chain(object):
    def __init__(self, path=''):
        self._path = path
    def __getattr__(self, path):
        return Chain('%s/%s' % (self._path, path))
    def __call__(self, path):
        return Chain('%s/%s' % (self._path, path))
    def __str__(self):
        return self._path

    __repr__ = __str__

In [52]:
Chain().user('hecc').repos

/user/hecc/repos